# FFT baby


In [ ]:
%matplotlib qt5
import numpy as np
from matplotlib import pyplot as plt

from typing import Callable

In [ ]:
def W_pow(N: int, sign: int = 1) -> Callable[[int], complex]:
    W = np.exp(2j*np.pi / N)
    memo = {}
    def _W_pow(nk: int):
        Wnk = memo.get(nk)
        if Wnk is None:
            Wnk = W**nk
            memo[nk] = Wnk
        return Wnk
    # precalculate
    for i in range(N):
        for j in range(N):
            _W_pow(sign*i*j)
    return _W_pow

def DFT(yn: np.ndarray) -> np.ndarray:
    "O(n^2)"
    N = yn.size
    W = W_pow(N)
    Ynk = np.vstack([yn] * N).astype(np.complex128)
    for n in range(N):
        for k in range(N):
            Ynk[n, k] *= W(n*k)
    Yn = np.sum(Ynk, axis=1)
    Yn *= (2*np.pi)**-0.5
    return Yn

def IDFT(Yn: np.ndarray) -> np.ndarray:
    "O(n^2)"
    N = Yn.size
    W = W_pow(N, sign=-1)
    ynk = np.vstack([Yn] * N).astype(np.complex128)
    for n in range(N):
        for k in range(N):
            ynk[n, k] *= W(-n*k)
    yn = np.sum(ynk, axis=1)
    yn *= (2*np.pi)**0.5 / N
    return yn

In [ ]:
def FFT(yn: np.ndarray) -> np.ndarray:
    N = yn.size
    W = np.exp(2j*np.pi / N)
    if N == 1:
        return W*yn
    if N % 2:
        raise ValueError("Amount of elements must be a power of 2")
    Yn = np.empty(N, dtype=np.complex128)
    Fe = FFT(yn[::2])
    Fo = FFT(yn[1::2])
    for n in range(N):
        Yn[n] = Fe[n % (N//2)] + W**n * Fo[n % (N//2)]
    Yn *= (2*np.pi)**-0.5
    return Yn
# TODO make this work


In [ ]:
def _f(x):
    if x < 1:
        return x
    return x - 2
f = np.vectorize(_f)


N = 64
x = np.linspace(0, 2, N)
y = f(x)
Yn = DFT(y)
yn = IDFT(Yn)

print(f"{np.allclose(y, yn) = }")

plt.figure()
plt.scatter(x, y)
plt.scatter(x, yn.real)
plt.legend(["y", "IDFT(DFT(y))"])
plt.title("Test back transform")

plt.figure()
plt.plot(Yn.real)
plt.plot(Yn.imag)
plt.legend(["$\\Re$", "$\\Im$"])
plt.title("Yn")
plt.show()

In [ ]:
def plot_Y(f: Callable[[np.ndarray], np.ndarray], N: int, a: float = 0, b: float = 2*np.pi):
    x = np.linspace(a, b, N)
    y = f(x)
    Yn = DFT(y)
    plt.figure()
    plt.plot(Yn.real)
    plt.plot(Yn.imag)
    plt.legend(["$\\Re$", "$\\Im$"])
    plt.title("Yn")
    plt.show()

In [ ]:
N = 8
plot_Y(np.sin, N)
plot_Y(np.cos, N)
plot_Y(lambda x: np.cos(x) + 3, N)
plot_Y(lambda x: np.cos(x + 5) + 3, N)

In [ ]:
N = 64
plot_Y(lambda x: np.exp(-x / (2*np.pi)), N)